In [ ]:
# imports

# dataset loading
# Notebook helper functions for using ZeekDataset
# Save this code into a notebook cell (or a .py file) and run it.



# Try to import ZeekDataset. If it's defined in the notebook, this import can be omitted.
from zeek_dataset import find_and_load_datasets, sample_10_from_each_dataset



In [ ]:
# paths, constants
root = "./dataset-private/"
validation_split=0.1
model_save_path = "./model.pkl"
experiment_name = "exp1"


## Loading, init

In [ ]:
loaders = find_and_load_datasets(root, prefix_regex=r"^\d{3}", data_subdir="data")
print("Found datasets:", list(loaders.keys()))
results = sample_10_from_each_dataset(loaders, seed=42)
for ds_name, info in results.items():
    print(f"Dataset: {ds_name}  (file used: {info['file']})  samples: {len(info['samples'])}")
    display(info['df'])

In [ ]:
# model init, loading from file (pickle)

## Pipeline

: 

In [ ]:
# final test?

## Reporting

In [ ]:
#save results, process reports into files!